## Anomaly Detection Business - 비즈니스 평가

* 참고 이미지

    <img src="https://i.pinimg.com/originals/3c/cd/c7/3ccdc785f2388f6e0ef9fc0ff561c7bc.jpg" width="500">

#### 가상 프로젝트 시나리오
- 비즈니스 배경
   - 펄프 및 제지 공장 데이터셋은 제지 산업의 생산 공정에서 발생하는 다양한 데이터를 포함하고 있음. 이러한 데이터셋은 제지 공장의 기계와 공정에서 수집된 센서 데이터, 운영 로그, 품질 측정 결과 등을 포함할 수 있음. 이 데이터를 활용하여 공정의 이상 징후를 탐지하고, 생산성을 높이며, 유지보수 비용을 절감하는 등의 작업을 수행할 수 있음.
   <br/>
- 구체적 비즈니스 상황
   - 한 롤로 종이를 말다가 찢어지는 사고가 하루에 한번 씩 발생함.
   - 이때마다 공정 중단 및 수율 저하 등의 이유로 평균 100만원 손실 발생.
   - 사전에 감지하는 것이 중요하지만 이를 예상하기는 어려움.
   - 5~10%만 감소 시킬수 있다면 상당한 비용절감 효과가 예상됨.
 <br/>   
- 본 프로젝트로 인한 기타 효익
   - 생산 효율성 증가: 이상 징후를 조기에 감지하여 공정 중단을 최소화하고, 생산성을 높임.
   - 품질 관리: 제품의 품질을 일관되게 유지하며, 품질 문제가 생산 초기에 발견되도록 함.
   - 비용 절감: 고장이나 비효율적인 운영으로 인한 비용을 줄임. 유지보수 작업을 계획적으로 수행하여 긴급 수리 비용과 다운타임을 줄일 수 있음.

- 데이터셋의 주요 구성 요소 (데이터셋의 필드는 마스킹 처리됨)
   - 센서 데이터: 기계의 온도, 압력, 속도, 전력 사용량 등을 실시간으로 모니터링하는 센서에서 수집된 데이터. 이러한 데이터는 기계의 정상 작동 범위를 벗어난 활동을 감지하는 데 중요함.
   - 운영 로그: 기계의 작동 시간, 고장 및 유지보수 기록, 작업자의 교대 시간표 등이 포함됨. 이 정보는 기계의 성능과 고장 패턴 분석에 유용함.
   - 품질 데이터: 완성된 제품의 두께, 무게, 강도 등 품질 관련 측정치. 제품 품질이 사양을 벗어났을 경우 이상 상태로 판단할 수 있음.
   - 환경 데이터: 작업 환경의 온도, 습도와 같은 조건을 기록한 데이터. 이는 생산 공정에 영향을 미칠 수 있는 중요한 변수임.
<br/>
- 데이터 구조
   - 15일간 18000건의 트랙잭션 시계열 데이터
   - 시간 간격은 2분
   - y : 0은 정상, 1은 비정상 데이터
   - x1 ~ x61 : 센서, 운영, 품질, 환경 데이터
<br/>
- 주요 고려사항
   - y가 1인 데이터는 장애 발생 시점이기 때문에, 사전예측을 위해서는 2-4분 전 예측이 가능하도록 데이터 Shift가 필요
   - t(n) 이 1(장애) 라면 t(n-2)와 t(n-1) 로 1로 값을 치환해주고, t(n)인 데이터는 행삭제 (장애 발생 시점 데이터를 이용해 다음을 예측하는 것은 이치에 맞지 않기 때문)
   - 비즈니스 제한사항 : 장애가 예측되면 속도를 출여 장애를 예방(100만원 손실 예방) 가능하지만, 생산성 저하로 속도 감소 1회 진행시 5만원 손실 발생

##### 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from lib.ae import build_dynamic_ae
from lib.vis import autoencoder_report

##### 데이터 로드

In [ ]:
# 공정 데이터 불러오기
data = pd.read_csv('dataset/pulp_and_paper_mill.csv')
data.head()

##### 이상치 분포 확인

In [ ]:
date_tmp = data.copy()
date_tmp['time'] = pd.to_datetime(data.time, format = '%m/%d/%y %H:%M')
date_tmp['day'] = date_tmp['time'].dt.day
date_tmp.loc[date_tmp['y']==1].groupby('day', as_index=False)['y'].count()

##### 데이터 전처리 (중요)

In [8]:
# 데이터 복사본 생성
new_data = data.copy()

# 이상치 감지를 위한 새로운 레이블 컬럼 초기화
new_data['new_y'] = 0

# 이상치(y=1)가 발생하기 전 2개 데이터 포인트에 레이블 지정
for i, row in new_data.iterrows():
    if row['y'] == 1:
        # 이상치 발생 2단계 전 데이터 포인트 레이블링
        if i > 1:
            new_data.loc[i-2, 'new_y'] = 1
        # 이상치 발생 1단계 전 데이터 포인트 레이블링
        if i > 0:
            new_data.loc[i-1, 'new_y'] = 1

# 실제 이상치 데이터(y=1) 제거
new_data = new_data[new_data['y'] != 1]

# 기존 레이블 컬럼 삭제
new_data.drop(['y'], axis=1, inplace=True)

# 새 레이블 컬럼 이름 변경
new_data.rename(columns={'new_y': 'y'}, inplace=True)

In [9]:
new_data.drop('time', axis=1, inplace=True)

In [ ]:
new_data.shape

In [11]:
train, test = train_test_split(new_data, test_size=4000)
train, val = train_test_split(train, test_size=4000)

In [12]:
x_train_0 = train.loc[train['y'] == 0].drop(['y'], axis=1)
x_val_0 = val.loc[val['y'] == 0].drop(['y'], axis=1)

In [13]:
# 스케일링 fitting
scaler = StandardScaler()

# 적용
x_train_0_s = scaler.fit_transform(x_train_0)
x_val_0_s = scaler.transform(x_val_0) #학습시 validation용

x_test_s = scaler.transform(test.drop(['y'], axis = 1)) # 모델 평가용.

#### 모델링

In [ ]:
input_dim = x_train_0_s.shape[1]
input_dim

In [ ]:
autoencoder = build_dynamic_ae(
    input_dim, 
    encode_layers=[32, 24, 16, 8],
    decode_layers=[16, 24, 32], 
    learning_rate=1e-3,
    loss='mse',
    metrics=['accuracy']
)

In [ ]:
history = autoencoder.fit(x_train_0_s, x_train_0_s, epochs=200, batch_size=32, validation_data=(x_val_0_s, x_val_0_s))

##### 모델평가 및 시각화

In [ ]:
x_pred = autoencoder.predict(x_test_s)

In [ ]:
autoencoder_report(x_test_s, x_pred, 1-test['y'], 0.5)

##### 모델평가 by 비즈니스 관점

|              | 정상 예측(0) | 장애 예측(1) |
|--------------|-------------|-------------|
| 정상 작동(0) | TN_cost = 0  | FP_cost = 5 |
| 장애 발생(1) | FN_cost = 100| TP_cost = 5 |

In [25]:
# 기대 손실비용 변수
TN_cost, FP_cost, FN_cost, TP_cost = 0, 5, 100, 5

In [ ]:
from sklearn.metrics import precision_recall_curve
mse = np.mean(np.power(x_test_s - x_pred, 2), axis=1)
_, _, thresholds = precision_recall_curve(test['y'], mse, pos_label=1)
thresholds

In [27]:
cost = []
for at in thresholds :
    y_pred = np.where(mse > at, 1, 0)
    TN, FP, FN, TP = confusion_matrix(test['y'], y_pred).ravel()
    cost.append(TP*TP_cost + FP*FP_cost + FN*FN_cost + TN*TN_cost)

cost = np.array(cost)
base_cost = test['y'].sum() * 100 # 실제 발생한 손실

In [ ]:
fig = px.line(x=thresholds, y=cost, title='임계값 대비 비용', labels={'x': '임계값', 'y': '비용'})
fig.add_hline(y=base_cost, line_color='red', line_width=2, line_dash='dash', annotation_text="기본 비용")
fig.show()

In [ ]:
best_threshold = thresholds[list(cost).index(cost.min())]
best_threshold
